In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 50000)
pd.set_option('display.max_rows', 1000000)
pd.set_option('display.max_colwidth',None)
pd.set_option('display.float_format',lambda x: '%.4f' % x)
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as pl
import plotly.express as px
import plotly.graph_objects as go
from matplotlib.pyplot import figure
import matplotlib.pylab as pll
from collections import Counter
from datetime import datetime
import time
import random
import math
import os
import itertools
from matplotlib import ticker
import matplotlib.colors as colors
from pylab import rcParams
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error,mean_squared_error, explained_variance_score,r2_score
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeClassifier

In [ ]:
slurm1 = pd.read_csv('SlurmPlot2.csv')
slurm1.tail(100)

In [ ]:
slurm = slurm1[['UID','JobID','Account','Submittimestamp','Start','End','Eligibletimestamp','Starttimestamp','Endtimestamp','AllocNodes','AllocCPUS','TimelimitRaw', 'NNodes','Submit','State','ExitCode','QOS','QOSRAW','Partition','TRESAllocMem','TRESINMaxUsagemem']]

In [ ]:
slurm = slurm[slurm["Starttimestamp"] != 'Unknown']
slurm = slurm[slurm["Endtimestamp"] != 'Unknown']
slurm['JobLen'] = slurm['Endtimestamp'].astype(float) - slurm['Starttimestamp'].astype(float)
slurm['Totalres'] = slurm['JobLen'] * slurm['NNodes']
slurm['JobID'] = slurm['JobID'].astype(str)

In [ ]:
discard = ["."]
waittime = slurm[~slurm.JobID.str.contains('|'.join(discard),regex = False)]
waittime['Jcount'] = 1
waittime['Waittime'] = waittime['Starttimestamp'].astype(float) - waittime['Submittimestamp'].astype(float)

In [ ]:
waittime['year'] = pd.DatetimeIndex(waittime['Submit']).year
waittime['month'] = pd.DatetimeIndex(waittime['Submit']).month
waittime['year'] = waittime['year'].apply(str)
waittime['month'] = waittime['month'].apply(str)
waittime['yearmonth'] = waittime['year'] + '-' + waittime['month']

In [ ]:
waittime.info()

In [ ]:
waittime = waittime.loc[waittime.JobLen > 0]

In [ ]:
waittime.head()

In [ ]:
prediction = waittime[['Account','TimelimitRaw','JobLen','NNodes','Partition','Jcount','Totalres']]

In [ ]:
prediction.info()

In [ ]:
def f(row):
    if row['Partition'] == 'batch':
        val = 1
    else:
        val = 0
    return val
prediction['batch'] = prediction.apply(f,axis =1)
def f1(row):
    if row['Partition'] == 'all':
        val = 1
    else:
        val = 0
    return val
prediction['All'] = prediction.apply(f1,axis =1)
def f2(row):
    if row['Partition'] == 'long':
        val = 1
    else:
        val = 0
    return val
prediction['long'] = prediction.apply(f2,axis =1)

def f3(row):
    if row['Partition'] == 'short':
        val = 1
    else:
        val = 0
    return val
prediction['short'] = prediction.apply(f3,axis =1)
def f4(row):
    if row['Partition'] == 'test':
        val = 1
    else:
        val = 0
    return val
prediction['test'] = prediction.apply(f4,axis =1)
def f5(row):
    if row['Partition'] == 'fpga':
        val = 1
    else:
        val = 0
    return val
prediction['fpga'] = prediction.apply(f5,axis =1)
def f6(row):
    if row['Partition'] == 'fpgasyn':
        val = 1
    else:
        val = 0
    return val
prediction['fpgasyn'] = prediction.apply(f6,axis =1)
def f7(row):
    if row['Partition'] == 'dgx':
        val = 1
    else:
        val = 0
    return val
prediction['dgx'] = prediction.apply(f7,axis =1)

In [ ]:
prediction.info()

In [ ]:
temp = prediction.groupby(['Account'], as_index=False).agg({'Jcount':'sum','batch':'sum','All':'sum',
                                                      'long':'sum','short':'sum','test':'sum','fpga':'sum','fpgasyn':'sum','dgx':'sum'})
temp = temp.sort_values('Account')
rcParams['figure.figsize'] = 12, 8
fig,ax = plt.subplots()
ax.grid(False)
ax.set_title("Number of jobs for every partition according to accounts ")
width = 1 
# ax.bar(temp.Account, temp.Jcount, width, color='yellow',alpha = 0.5)
ax.bar(temp.Account, temp.batch, width, color='green',alpha = 0.5)
ax.bar(temp.Account, temp.All, width,bottom =temp.batch,  color='m',alpha = 0.5)
ax.bar(temp.Account, temp.long, width,bottom =  temp.batch + temp.All,  color='brown',alpha = 0.5)
ax.bar(temp.Account, temp.short, width,bottom = temp.batch + temp.All + temp.long,  color='r',alpha = 0.5)
ax.bar(temp.Account, temp.test, width,bottom = temp.batch + temp.All + temp.long + temp.short , color='yellow',alpha = 0.5)
ax.bar(temp.Account, temp.fpga, width, bottom =  temp.batch + temp.All + temp.long + temp.short + temp.test ,color='k',alpha = 0.5)       
ax.bar(temp.Account, temp.fpgasyn, width, bottom = temp.batch + temp.All + temp.long + temp.short + temp.test + temp.fpga ,color='blue',alpha = 0.5) 
ax.bar(temp.Account, temp.dgx, width, bottom = temp.batch + temp.All + temp.long + temp.short + temp.test + temp.fpga + temp.fpgasyn ,color='cyan',alpha = 0.5) 
ax.set_xlabel('Account', fontsize=16)
ax.set_ylabel('Number of jobs', fontsize=16)
plt.subplots_adjust(left=0, bottom=0)
plt.legend(['batch','all','long','short','test','fpga','fpgasyn','dgx' ])
plt.show()
if not os.path.exists('classification'):
    os.makedirs('classification')
fig.savefig('classification/statistics.png',bbox_inches='tight', dpi=100)

In [ ]:
temp = prediction.groupby('Partition', as_index=False).agg({'Jcount':'sum'})
rcParams['figure.figsize'] = 10, 10
temp['Partition'] = temp['Partition'].apply(str)
plots = sns.barplot(x ='Partition',y = 'Jcount',data =  temp)
plots.set_title('Number of jobs according to different job Partition',size = 22)
plots.set_xlabel("Partition", fontsize = 16)
plots.set_ylabel("Number of jobs", fontsize = 16)
for bar in plots.patches:
    plots.annotate(int(bar.get_height()),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=16, xytext=(0, 8),
                   textcoords='offset points')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
if not os.path.exists('classification'):
    os.makedirs('classification')
plt.savefig('classification/Job_PARTITION_1.png',bbox_inches='tight', dpi=100)

In [ ]:
prediction.head()

In [ ]:
prediction['Timelimit'] = prediction['TimelimitRaw'] * 60

In [ ]:
regression = prediction[['Account','NNodes','Partition','Timelimit','JobLen','Totalres']]

In [ ]:
regression.head()

## Decision Tree and Random Forest Regression for all the records

In [ ]:
X = regression.iloc[:,0:4]
y = regression.iloc[:,4:6]

In [ ]:
X.head()

In [ ]:
label_encoder = preprocessing.LabelEncoder()
X['Partition']= label_encoder.fit_transform(X['Partition'])
X['Partition'].unique()

In [ ]:
y.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=X['Partition'], test_size=0.25,random_state = 42)

In [ ]:
y_train_dummy = y_train.copy()

In [ ]:
y_train = y_train[['JobLen']]

In [ ]:
y_train.head()

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
dt = DecisionTreeRegressor(random_state = 42).fit(X_train,y_train)

In [ ]:
rf = RandomForestRegressor(random_state = 42).fit(X_train,y_train)

In [ ]:
print('coefficient r_sq for decision tree is %.2f'% dt.score(X_train,y_train))
print('coefficient r_sq for random forest is %.2f'% rf.score(X_train,y_train))

In [ ]:
ytrain_pred = dt.predict(X_train)
ytrain_pred2 = rf.predict(X_train)

In [ ]:
y_train.head()

In [ ]:
print('coefficient r_sq for dt is ', r2_score(y_train,ytrain_pred))
print('mean absolute error for dt is ', mean_absolute_error(y_train,ytrain_pred))
print('mean squared error for dt is ', mean_squared_error(y_train,ytrain_pred))
print('explained variance score for dt is %.2f', explained_variance_score(y_train,ytrain_pred))

In [ ]:
print('coefficient r_sq for rf is ', r2_score(y_train,ytrain_pred2))
print('mean absolute error for rf is ', mean_absolute_error(y_train,ytrain_pred2))
print('mean squared error for rf is ', mean_squared_error(y_train,ytrain_pred2))
print('explained variance score for rf is %.2f', explained_variance_score(y_train,ytrain_pred2))

In [ ]:
ytest_pred = dt.predict(X_test)
ytest_pred2 = rf.predict(X_test)

In [ ]:
X_test.shape

In [ ]:
y_test_dummy = y_test.copy()

In [ ]:
y_test = y_test[['JobLen']]

In [ ]:
print('coefficient r_sq for decision tree is %.2f'% dt.score(X_test,y_test))
print('coefficient r_sq for random forest is %.2f'% rf.score(X_test,y_test))

In [ ]:
print('coefficient r_sq for decision tree is %.2f'% r2_score(y_test,ytest_pred))
print('coefficient r_sq for random forest is %.2f'% r2_score(y_test,ytest_pred2))

In [ ]:
rcParams['figure.figsize'] = 100, 20
x_ax = range(len(X_test))
plt.plot(x_ax,y_test,label = 'actual',color = 'k',linestyle = '-')
plt.scatter(x_ax,ytest_pred,label = 'predicted',color = 'r',linestyle = '--')
if not os.path.exists('classification'):
    os.makedirs('classification')
plt.savefig('classification/dt.png')

In [ ]:
rcParams['figure.figsize'] = 100, 20
x_ax = range(len(X_test))
plt.plot(x_ax,y_test,label = 'actual',color = 'k',linestyle = '-')
plt.scatter(x_ax,ytest_pred2,label = 'predicted',color = 'r',linestyle = '--')
if not os.path.exists('classification'):
    os.makedirs('classification')
plt.savefig('classification/rf.png')

In [ ]:
y_test_dummy.head(20)

In [ ]:
y_test_dummy['Predicted_dt'] = ytest_pred
y_test_dummy['Predicted_rf'] = ytest_pred2

In [ ]:
rcParams['figure.figsize'] = 15, 15
import matplotlib.pyplot as plt
mm=21*24*3600
fig = plt.figure()
ax = fig.add_subplot(111)
heatmap, xedges, yedges = np.histogram2d(x=y_test_dummy['Predicted_dt'],y=y_test_dummy['JobLen'],weights=y_test_dummy['Totalres'] ,bins=200,range=[[0,mm],[0,mm]])
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
print(heatmap.min(),heatmap.max())
cutoff=10
pcm=plt.pcolormesh(xedges, yedges, heatmap, cmap='inferno',norm=colors.LogNorm(vmin=heatmap.min()+cutoff, vmax=heatmap.max()+cutoff))
plt.xlabel('Actual run time in seconds')
plt.ylabel('Predicted run time in seconds')
plt.xlim(0,mm)
plt.ylim(0,mm)
plt.axis('square')
plt.title('Decision tree : 2D histogram for the predicted vs. actual run time')
fig.colorbar(pcm, ax=ax, extend='max')
if not os.path.exists('classification'):
    os.makedirs('classification')
plt.savefig('classification/dt_new.png')

In [ ]:
rcParams['figure.figsize'] = 15, 15
import matplotlib.pyplot as plt
mm=21*24*3600
fig = plt.figure()
ax = fig.add_subplot(111)
heatmap, xedges, yedges = np.histogram2d(x=y_test_dummy['Predicted_rf'],y=y_test_dummy['JobLen'],weights=y_test_dummy['Totalres'] ,bins=200,range=[[0,mm],[0,mm]])
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
print(heatmap.min(),heatmap.max())
cutoff=10
pcm=plt.pcolormesh(xedges, yedges, heatmap, cmap='inferno',norm=colors.LogNorm(vmin=heatmap.min()+cutoff, vmax=heatmap.max()+cutoff))
plt.xlabel('Actual run time in seconds')
plt.ylabel('Predicted run time in seconds')
plt.xlim(0,mm)
plt.ylim(0,mm)
plt.axis('square')
plt.title('Random forest : 2D histogram for the predicted vs. actual run time')
fig.colorbar(pcm, ax=ax, extend='max')
if not os.path.exists('classification'):
    os.makedirs('classification')
plt.savefig('classification/rf_new.png')

### Hyper - parameter tuning

In [ ]:
# n_estimators = [5,20,50,100,200,300,500] # number of trees in the random forest
# max_features = ['auto', 'sqrt'] # number of features in consideration at every split
# max_depth = [int(x) for x in np.linspace(10, 120, num = 12)] # maximum number of levels allowed in each decision tree
# min_samples_split = [2, 6, 10,16,20] # minimum sample number to split a node
# min_samples_leaf = [1, 3, 4,6] # minimum sample number that can be stored in a leaf node
# bootstrap = [True, False] # method used to sample data points

# random_grid = {'n_estimators': n_estimators,

# 'max_features': max_features,

# 'max_depth': max_depth,

# 'min_samples_split': min_samples_split,

# 'min_samples_leaf': min_samples_leaf,

# 'bootstrap': bootstrap}

In [ ]:
# ## Importing Random Forest Classifier from the sklearn.ensemble
# from sklearn.ensemble import RandomForestRegressor
# rff = RandomForestRegressor()

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# rf_random = RandomizedSearchCV(estimator = rff,param_distributions = random_grid,
#                n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [ ]:
# rf_random.fit(X_train, y_train)

In [ ]:
# print ('Random grid: ', random_grid, '\n')
# # print the best parameters
# print ('Best Parameters: ', rf_random.best_params_, ' \n')

In [ ]:
# rf = RandomForestRegressor(n_estimators = 300, min_samples_split = 16, min_samples_leaf = 1, max_features = 'sqrt', max_depth = 100, bootstrap = False,  random_state = 42).fit(X_train,y_train)

In [ ]:
# print('coefficient r_sq for random forest is %.2f'% rf.score(X_train,y_train))

In [ ]:

# ytrain_pred2 = rf.predict(X_train)

In [ ]:
# print('coefficient r_sq for rf is ', r2_score(y_train,ytrain_pred2))
# print('mean absolute error for rf is ', mean_absolute_error(y_train,ytrain_pred2))
# print('mean squared error for rf is ', mean_squared_error(y_train,ytrain_pred2))
# print('explained variance score for rf is %.2f', explained_variance_score(y_train,ytrain_pred2))

In [ ]:

# ytest_pred2 = rf.predict(X_test)

In [ ]:

# print('coefficient r_sq for random forest is %.2f'% rf.score(X_test,y_test))

In [ ]:

# print('coefficient r_sq for random forest is %.2f'% r2_score(y_test,ytest_pred2))

In [ ]:
# rcParams['figure.figsize'] = 100, 20
# x_ax = range(len(X_test))
# plt.plot(x_ax,y_test,label = 'actual',color = 'k',linestyle = '-')
# plt.scatter(x_ax,ytest_pred2,label = 'predicted',color = 'r',linestyle = '--')
# if not os.path.exists('classification'):
#     os.makedirs('classification')
# plt.savefig('classification/rf_hyper.png')

In [ ]:
# rcParams['figure.figsize'] = 100, 20
# plt.scatter(y_test,ytest_pred2,label = 'actual vs predicted',color = 'r')
# if not os.path.exists('classification'):
#     os.makedirs('classification')
# plt.savefig('classification/rf_new_hyper.png')

### Only for Completed jobs 

In [ ]:
prediction2 = waittime[['Account','TimelimitRaw','JobLen','NNodes','Partition','Jcount','State','Totalres']]

In [ ]:
prediction2 = prediction2.loc[(prediction2['State'] == 'COMPLETED')]

In [ ]:
prediction2.shape

In [ ]:
regression2 = prediction2[['Account','NNodes','Partition','TimelimitRaw','JobLen','Totalres']]

In [ ]:
regression2['TimelimitRaw'] = regression2['TimelimitRaw'] * 60

In [ ]:
X = regression2.iloc[:,0:4]
y = regression2.iloc[:,4:6]

In [ ]:
X.head()

In [ ]:
label_encoder = preprocessing.LabelEncoder()
X['Partition']= label_encoder.fit_transform(X['Partition'])
X['Partition'].unique()

In [ ]:
y.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=X['Partition'], test_size=0.25,random_state = 42)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
y_train_dummy = y_train.copy()

In [ ]:
y_train = y_train[['JobLen']]

In [ ]:
# y_train = y_train['JobLen']

In [ ]:
dt = DecisionTreeRegressor(random_state = 42).fit(X_train,y_train)

In [ ]:
rf = RandomForestRegressor(random_state = 42).fit(X_train,y_train)

In [ ]:
print('coefficient r_sq for decision tree is %.2f'% dt.score(X_train,y_train))
print('coefficient r_sq for random forest is %.2f'% rf.score(X_train,y_train))

In [ ]:
ytrain_pred = dt.predict(X_train)
ytrain_pred2 = rf.predict(X_train)

In [ ]:
print('coefficient r_sq for dt is ', r2_score(y_train,ytrain_pred))
print('mean absolute error for dt is ', mean_absolute_error(y_train,ytrain_pred))
print('mean squared error for dt is ', mean_squared_error(y_train,ytrain_pred))
print('explained variance score for dt is %.2f', explained_variance_score(y_train,ytrain_pred))

In [ ]:
print('coefficient r_sq for rf is ', r2_score(y_train,ytrain_pred2))
print('mean absolute error for rf is ', mean_absolute_error(y_train,ytrain_pred2))
print('mean squared error for rf is ', mean_squared_error(y_train,ytrain_pred2))
print('explained variance score for rf is %.2f', explained_variance_score(y_train,ytrain_pred2))

In [ ]:
ytest_pred = dt.predict(X_test)
ytest_pred2 = rf.predict(X_test)

In [ ]:
X_test.shape

In [ ]:
y_test_dummy = y_test.copy()

In [ ]:
y_test = y_test[['JobLen']]

In [ ]:
print('coefficient r_sq for decision tree is %.2f'% dt.score(X_test,y_test))
print('coefficient r_sq for random forest is %.2f'% rf.score(X_test,y_test))

In [ ]:
print('coefficient r_sq for decision tree is %.2f'% r2_score(y_test,ytest_pred))
print('coefficient r_sq for random forest is %.2f'% r2_score(y_test,ytest_pred2))

In [ ]:
y.head()

In [ ]:
rcParams['figure.figsize'] = 100, 20
x_ax = range(len(X_test))
plt.plot(x_ax,y_test,label = 'actual',color = 'k',linestyle = '-')
plt.scatter(x_ax,ytest_pred,label = 'predicted',color = 'r',linestyle = '--')
if not os.path.exists('classification'):
    os.makedirs('classification')
plt.savefig('classification/dt2.png')

In [ ]:
rcParams['figure.figsize'] = 100, 20
x_ax = range(len(X_test))
plt.plot(x_ax,y_test,label = 'actual',color = 'k',linestyle = '-')
plt.scatter(x_ax,ytest_pred2,label = 'predicted',color = 'r',alpha = 0.5,linestyle = '--')
if not os.path.exists('classification'):
    os.makedirs('classification')
plt.savefig('classification/rf2.png')

In [ ]:
y_test_dummy['Predicted_dt'] = ytest_pred
y_test_dummy['Predicted_rf'] = ytest_pred2

In [ ]:
rcParams['figure.figsize'] = 15, 15
import matplotlib.pyplot as plt
mm=21*24*3600
fig = plt.figure()
ax = fig.add_subplot(111)
heatmap, xedges, yedges = np.histogram2d(x=y_test_dummy['Predicted_dt'],y=y_test_dummy['JobLen'],weights=y_test_dummy['Totalres'] ,bins=200,range=[[0,mm],[0,mm]])
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
print(heatmap.min(),heatmap.max())
cutoff=10
pcm=plt.pcolormesh(xedges, yedges, heatmap, cmap='inferno',norm=colors.LogNorm(vmin=heatmap.min()+cutoff, vmax=heatmap.max()+cutoff))
plt.xlabel('Actual run time in seconds')
plt.ylabel('Predicted run time in seconds')
plt.xlim(0,mm)
plt.ylim(0,mm)
plt.axis('square')
plt.title('Decision tree : 2D histogram for the predicted vs. actual run time')
fig.colorbar(pcm, ax=ax, extend='max')
if not os.path.exists('classification'):
    os.makedirs('classification')
plt.savefig('classification/dt2_new.png')

In [ ]:
rcParams['figure.figsize'] = 15, 15
import matplotlib.pyplot as plt
mm=21*24*3600
fig = plt.figure()
ax = fig.add_subplot(111)
heatmap, xedges, yedges = np.histogram2d(x=y_test_dummy['Predicted_rf'],y=y_test_dummy['JobLen'],weights=y_test_dummy['Totalres'] ,bins=200,range=[[0,mm],[0,mm]])
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
print(heatmap.min(),heatmap.max())
cutoff=10
pcm=plt.pcolormesh(xedges, yedges, heatmap, cmap='inferno',norm=colors.LogNorm(vmin=heatmap.min()+cutoff, vmax=heatmap.max()+cutoff))
plt.xlabel('Actual run time in seconds')
plt.ylabel('Predicted run time in seconds')
plt.xlim(0,mm)
plt.ylim(0,mm)
plt.axis('square')
plt.title('Random forest : 2D histogram for the predicted vs. actual run time')
fig.colorbar(pcm, ax=ax, extend='max')
if not os.path.exists('classification'):
    os.makedirs('classification')
plt.savefig('classification/rf2_new.png')

In [ ]:
y_test1 = y_test.copy()

In [ ]:
# X_test['Predicted'] = ytest_pred2

In [ ]:
y_test1['predicted'] = ytest_pred2
y_test1['Timelimit'] = X_test['TimelimitRaw'].to_numpy()

In [ ]:
y_test1.info()

In [ ]:
i = 0
predicted = []
JobLen = []
timelimit = []
df = pd.DataFrame(columns = ['Predicted','JobLen','timelimit'])
for index, row in y_test1.iterrows():
    if row['predicted'] < row['JobLen']:
        i = i + 1
        predicted.append(row['predicted'])
        JobLen.append(row['JobLen'])
        timelimit.append(row['Timelimit'])

In [ ]:
df['Predicted'] = predicted
df['JobLen'] = JobLen
df['timelimit'] = timelimit


In [ ]:
print(i)
df.head(10)

In [ ]:
y_train2 = y_train.copy()

In [ ]:
X_train.head()

In [ ]:
y_train2['Account'] = X_train['Account'].to_numpy()
y_train2['NNodes'] = X_train['NNodes'].to_numpy()
y_train2['Partition'] = X_train['Partition'].to_numpy()
y_train2['Timelimit'] = X_train['TimelimitRaw'].to_numpy()
y_train2['predicted'] = ytrain_pred2

In [ ]:
i = 0
predicted1 = []
JobLen1 = []
timelimit1 = []
account = []
nnodes = []
partition = []
df1 = pd.DataFrame(columns = ['Account','NNodes','Partition','TimelimitRaw','Predicted','JobLen'])
for index, row in y_train2.iterrows():
    if row['predicted'] < row['JobLen']:
        i = i + 1
        account.append(row['Account'])
        nnodes.append(row['NNodes'])
        partition.append(row['Partition'])
        predicted1.append(row['predicted'])
        JobLen1.append(row['JobLen'])
        timelimit1.append(row['Timelimit'])

In [ ]:
df1['Account'] = account
df1['NNodes'] = nnodes
df1['Partition'] = partition
df1['Predicted'] = predicted1
df1['JobLen'] = JobLen1
df1['TimelimitRaw'] = timelimit1

In [ ]:
print(i)
df1.head()

In [ ]:
xtrain = df1.iloc[:,0:4]
ytrain = df1.iloc[:,4:5]

In [ ]:
ytrain.head()

In [ ]:
xy_test = pd.concat([X_test, y_test], axis="columns")

In [ ]:
xy_test.head(10)

In [ ]:
xy_test = xy_test.sample(frac=0.30)

In [ ]:
xy_test.info()

In [ ]:
X_test = xy_test[['Account','NNodes','Partition','TimelimitRaw']]

In [ ]:
y_test.head()

In [ ]:
y_test = xy_test[['JobLen']]

In [ ]:
dtt = DecisionTreeRegressor(random_state = 42).fit(xtrain,ytrain)

In [ ]:
rff = RandomForestRegressor(random_state = 42).fit(xtrain,ytrain)

In [ ]:
print('coefficient r_sq for decision tree is %.2f'% dtt.score(xtrain,ytrain))
print('coefficient r_sq for random forest is %.2f'% rff.score(xtrain,ytrain))

In [ ]:
ytrainn_pred = dtt.predict(xtrain)
ytrainn_pred2 = rff.predict(xtrain)

In [ ]:
print('coefficient r_sq for dt is ', r2_score(ytrain,ytrainn_pred))
print('mean absolute error for dt is ', mean_absolute_error(ytrain,ytrainn_pred))
print('mean squared error for dt is ', mean_squared_error(ytrain,ytrainn_pred))
print('explained variance score for dt is %.2f', explained_variance_score(ytrain,ytrainn_pred))

In [ ]:
print('coefficient r_sq for rf is ', r2_score(ytrain,ytrainn_pred2))
print('mean absolute error for rf is ', mean_absolute_error(ytrain,ytrainn_pred2))
print('mean squared error for rf is ', mean_squared_error(ytrain,ytrainn_pred2))
print('explained variance score for rf is %.2f', explained_variance_score(ytrain,ytrainn_pred2))

In [ ]:
X_test.head()

In [ ]:
ytest_predd = dtt.predict(X_test)
ytest_predd2 = rff.predict(X_test)

In [ ]:
y_test.info()

In [ ]:
print('coefficient r_sq for decision tree is %.2f'% dtt.score(X_test,y_test))
print('coefficient r_sq for random forest is %.2f'% rff.score(X_test,y_test))

In [ ]:
y_test2 = y_test.copy()

In [ ]:
y_test2.head()

In [ ]:
y_test2['Account'] = X_test['Account'].to_numpy()
y_test2['NNodes'] = X_test['NNodes'].to_numpy()
y_test2['Partition'] = X_test['Partition'].to_numpy()
y_test2['Timelimit'] = X_test['TimelimitRaw'].to_numpy()
y_test2['predicted'] = ytest_predd2

In [ ]:
i = 0
predicted1 = []
JobLen1 = []
timelimit1 = []
account = []
nnodes = []
partition = []
df2 = pd.DataFrame(columns = ['Account','NNodes','Partition','timelimit','Predicted','JobLen'])
for index, row in y_test2.iterrows():
    if row['predicted'] < row['JobLen']:
        i = i + 1
        account.append(row['Account'])
        nnodes.append(row['NNodes'])
        partition.append(row['Partition'])
        predicted1.append(row['predicted'])
        JobLen1.append(row['JobLen'])
        timelimit1.append(row['Timelimit'])

In [ ]:
df2['Account'] = account
df2['NNodes'] = nnodes
df2['Partition'] = partition
df2['Predicted'] = predicted1
df2['JobLen'] = JobLen1
df2['timelimit'] = timelimit1

In [ ]:
print(i)
df2.head()

In [ ]:
# n_estimators = [5,20,50,100,200,300,500] # number of trees in the random forest
# max_features = ['auto', 'sqrt'] # number of features in consideration at every split
# max_depth = [int(x) for x in np.linspace(10, 120, num = 12)] # maximum number of levels allowed in each decision tree
# min_samples_split = [2, 6, 10,16,20] # minimum sample number to split a node
# min_samples_leaf = [1, 3, 4,6] # minimum sample number that can be stored in a leaf node
# bootstrap = [True, False] # method used to sample data points

# random_grid = {'n_estimators': n_estimators,

# 'max_features': max_features,

# 'max_depth': max_depth,

# 'min_samples_split': min_samples_split,

# 'min_samples_leaf': min_samples_leaf,

# 'bootstrap': bootstrap}

In [ ]:
# ## Importing Random Forest Classifier from the sklearn.ensemble
# from sklearn.ensemble import RandomForestRegressor
# rff = RandomForestRegressor()

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# rf_random = RandomizedSearchCV(estimator = rff,param_distributions = random_grid,
#                n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [ ]:
# rf_random.fit(X_train, y_train)

In [ ]:
# print ('Random grid: ', random_grid, '\n')
# # print the best parameters
# print ('Best Parameters: ', rf_random.best_params_, ' \n')

In [ ]:
# rf = RandomForestRegressor(n_estimators = 300, min_samples_split = 16, min_samples_leaf = 1, max_features = 'sqrt', max_depth = 100, bootstrap = False,  random_state = 42).fit(X_train,y_train)

In [ ]:
# print('coefficient r_sq for random forest is %.2f'% rf.score(X_train,y_train))

In [ ]:
# ytrain_pred2 = rf.predict(X_train)

In [ ]:
# print('coefficient r_sq for rf is ', r2_score(y_train,ytrain_pred2))
# print('mean absolute error for rf is ', mean_absolute_error(y_train,ytrain_pred2))
# print('mean squared error for rf is ', mean_squared_error(y_train,ytrain_pred2))
# print('explained variance score for rf is %.2f', explained_variance_score(y_train,ytrain_pred2))

In [ ]:
# ytest_pred2 = rf.predict(X_test)

In [ ]:
# print('coefficient r_sq for random forest is %.2f'% rf.score(X_test,y_test))

In [ ]:
# print('coefficient r_sq for random forest is %.2f'% r2_score(y_test,ytest_pred2))

In [ ]:
# y.head()

In [ ]:
# rcParams['figure.figsize'] = 100, 20
# x_ax = range(len(X_test))
# plt.plot(x_ax,y_test,label = 'actual',color = 'k',linestyle = '-')
# plt.scatter(x_ax,ytest_pred2,label = 'predicted',color = 'r',alpha = 0.5,linestyle = '--')
# if not os.path.exists('classification'):
#     os.makedirs('classification')
# plt.savefig('classification/rf2_hyper.png')

In [ ]:
# rcParams['figure.figsize'] = 100, 20
# plt.scatter(y_test,ytest_pred2,label = 'actual vs predicted',color = 'r',alpha = 0.5)
# if not os.path.exists('classification'):
#     os.makedirs('classification')
# plt.savefig('classification/rf2_new_hyper.png')

## Feature Extraction

In [ ]:
Features = regression2[['Account','NNodes','Partition','TimelimitRaw','JobLen','Totalres']]

In [ ]:
Features.head()

In [ ]:
Features['TimelimitRaw'] = Features['TimelimitRaw'] / 60 
Features['JobLen'] = Features['JobLen'] / 60 

In [ ]:
Features['Extra'] = Features['TimelimitRaw'] - Features['JobLen']

In [ ]:
X = Features[['Account','NNodes','Partition','TimelimitRaw']]
y = Features[['Extra','Totalres']]


In [ ]:
X.head()

In [ ]:
label_encoder = preprocessing.LabelEncoder()
X['Partition']= label_encoder.fit_transform(X['Partition'])
X['Partition'].unique()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30,random_state = 42)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
y_train_dummy = y_train.copy()

In [ ]:
y_train = y_train[['Extra']]

In [ ]:
dt = DecisionTreeRegressor(random_state = 42).fit(X_train,y_train)

In [ ]:
rf = RandomForestRegressor(random_state = 42).fit(X_train,y_train)

In [ ]:
print('coefficient r_sq for decision tree is %.2f'% dt.score(X_train,y_train))
print('coefficient r_sq for random forest is %.2f'% rf.score(X_train,y_train))

In [ ]:
print('Adjusted r_sq dt',1 - (1-dt.score(X_train, y_train))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1))
print('Adjusted r_sq rf',1 - (1-rf.score(X_train, y_train))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1))

In [ ]:
ytrain_pred = dt.predict(X_train)
ytrain_pred2 = rf.predict(X_train)

In [ ]:
print('coefficient r_sq for dt is ', r2_score(y_train,ytrain_pred))
print('mean absolute error for dt is ', mean_absolute_error(y_train,ytrain_pred))
print('mean squared error for dt is ', mean_squared_error(y_train,ytrain_pred))
print('explained variance score for dt is %.2f', explained_variance_score(y_train,ytrain_pred))

In [ ]:
print('coefficient r_sq for rf is ', r2_score(y_train,ytrain_pred2))
print('mean absolute error for rf is ', mean_absolute_error(y_train,ytrain_pred2))
print('mean squared error for rf is ', mean_squared_error(y_train,ytrain_pred2))
print('explained variance score for rf is %.2f', explained_variance_score(y_train,ytrain_pred2))

In [ ]:
ytest_pred = dt.predict(X_test)
ytest_pred2 = rf.predict(X_test)

In [ ]:
X_test.shape

In [ ]:
y_test_dummy = y_test.copy()

In [ ]:
y_test = y_test[['Extra']]

In [ ]:
print('coefficient r_sq for decision tree is %.2f'% dt.score(X_test,y_test))
print('coefficient r_sq for random forest is %.2f'% rf.score(X_test,y_test))

In [ ]:
print('coefficient r_sq for decision tree is %.2f'% r2_score(y_test,ytest_pred))
print('coefficient r_sq for random forest is %.2f'% r2_score(y_test,ytest_pred2))

In [ ]:
print('Adjusted r_sq dt',1 - (1-dt.score(X_test, y_test))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1))
print('Adjusted r_sq rf',1 - (1-rf.score(X_test, y_test))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1))

In [ ]:
rcParams['figure.figsize'] = 100, 20
x_ax = range(len(X_test))
plt.plot(x_ax,y_test,label = 'actual',color = 'k',linestyle = '-')
plt.scatter(x_ax,ytest_pred,label = 'predicted',color = 'r',linestyle = '--')
if not os.path.exists('classification'):
    os.makedirs('classification')
plt.savefig('classification/dt3_newfeatures.png')

In [ ]:
rcParams['figure.figsize'] = 100, 20
x_ax = range(len(X_test))
plt.plot(x_ax,y_test,label = 'actual',color = 'k',linestyle = '-')
plt.scatter(x_ax,ytest_pred2,label = 'predicted',color = 'r',alpha = 0.5,linestyle = '--')
if not os.path.exists('classification'):
    os.makedirs('classification')
plt.savefig('classification/rf3_newfeatures.png')

In [ ]:
y_test_dummy['Predicted_dt'] = ytest_pred
y_test_dummy['Predicted_rf'] = ytest_pred2

In [ ]:
y_test_dummy.describe()

In [ ]:
rcParams['figure.figsize'] = 15, 15
import matplotlib.pyplot as plt
mm=21*24*60
fig = plt.figure()
ax = fig.add_subplot(111)
heatmap, xedges, yedges = np.histogram2d(x=y_test_dummy['Predicted_dt'],y=y_test_dummy['Extra'],weights=y_test_dummy['Totalres'] ,bins=200,range=[[0,mm],[0,mm]])
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
print(heatmap.min(),heatmap.max())
cutoff=10
pcm=plt.pcolormesh(xedges, yedges, heatmap, cmap='inferno',norm=colors.LogNorm(vmin=heatmap.min()+cutoff, vmax=heatmap.max()+cutoff))
plt.xlabel('Real timelimit and actual job length difference in seconds')
plt.ylabel('Predicted timelimit and actual job length difference in seconds')
plt.xlim(0,mm)
plt.ylim(0,mm)
plt.axis('square')
plt.title('Decision tree : 2D histogram for the predicted vs. actual run time')
fig.colorbar(pcm, ax=ax, extend='max')
if not os.path.exists('classification'):
    os.makedirs('classification')
plt.savefig('classification/dt3_new_features.png')

In [ ]:
rcParams['figure.figsize'] = 15, 15
import matplotlib.pyplot as plt
mm=21*24*60
fig = plt.figure()
ax = fig.add_subplot(111)
heatmap, xedges, yedges = np.histogram2d(x=y_test_dummy['Predicted_rf'],y=y_test_dummy['Extra'],weights=y_test_dummy['Totalres'] ,bins=200,range=[[0,mm],[0,mm]])
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
print(heatmap.min(),heatmap.max())
cutoff=10
pcm=plt.pcolormesh(xedges, yedges, heatmap, cmap='inferno',norm=colors.LogNorm(vmin=heatmap.min()+cutoff, vmax=heatmap.max()+cutoff))
plt.xlabel('Real timelimit and actual job length difference in seconds')
plt.ylabel('Predicted timelimit and actual job length difference in seconds')
plt.xlim(0,mm)
plt.ylim(0,mm)
plt.axis('square')
plt.title('Random forest : 2D histogram for the predicted vs. actual run time')
fig.colorbar(pcm, ax=ax, extend='max')
if not os.path.exists('classification'):
    os.makedirs('classification')
plt.savefig('classification/rf3_new_features.png')

In [ ]:
y_test['predicted'] = ytest_pred2
y_test['Timelimit'] = X_test['TimelimitRaw'].to_numpy()

In [ ]:
i = 0
predicted = []
extra = []
timelimit = []
df = pd.DataFrame(columns = ['Predicted','Extra','timelimit'])
for index, row in y_test.iterrows():
    if row['predicted'] > row['Extra']:
        i = i + 1
        predicted.append(row['predicted'])
        extra.append(row['Extra'])
        timelimit.append(row['Timelimit'])

In [ ]:
X_test.info()

In [ ]:
df['Predicted'] = predicted
df['Extra'] = extra
df['timelimit'] = timelimit


In [ ]:
print(i)
df.head(20)

### Manipulation the target

In [ ]:
Features = regression2[['Account','NNodes','Partition','TimelimitRaw','JobLen','Totalres']]

In [ ]:
Features.head()

In [ ]:
Features['TimelimitRaw'] = Features['TimelimitRaw'] / 60 
Features['JobLen'] = Features['JobLen'] / 60 

In [ ]:
Features['Extra'] = Features['TimelimitRaw'] - Features['JobLen']

In [ ]:
Features.describe()

In [ ]:
X = Features[['Account','NNodes','Partition','TimelimitRaw']]
y = Features[['Extra','Totalres']]


In [ ]:
y['Actual_Extra'] = y['Extra']

In [ ]:
y['Extra'] = (y['Extra'] - ((y['Extra'] * 30) / 100))

In [ ]:
y.head()

In [ ]:
label_encoder = preprocessing.LabelEncoder()
X['Partition']= label_encoder.fit_transform(X['Partition'])
X['Partition'].unique()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30,random_state = 42)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
y_train_dummy = y_train.copy()

In [ ]:
y_train = y_train[['Extra']]

In [ ]:
dt = DecisionTreeRegressor(random_state = 42).fit(X_train,y_train)

In [ ]:
rf = RandomForestRegressor(random_state = 42).fit(X_train,y_train)

In [ ]:
print('coefficient r_sq for decision tree is %.2f'% dt.score(X_train,y_train))
print('coefficient r_sq for random forest is %.2f'% rf.score(X_train,y_train))

In [ ]:
print('Adjusted r_sq dt',1 - (1-dt.score(X_train, y_train))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1))
print('Adjusted r_sq rf',1 - (1-rf.score(X_train, y_train))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1))

In [ ]:
ytrain_pred = dt.predict(X_train)
ytrain_pred2 = rf.predict(X_train)

In [ ]:
print('coefficient r_sq for dt is ', r2_score(y_train,ytrain_pred))
print('mean absolute error for dt is ', mean_absolute_error(y_train,ytrain_pred))
print('mean squared error for dt is ', mean_squared_error(y_train,ytrain_pred))
print('explained variance score for dt is %.2f', explained_variance_score(y_train,ytrain_pred))

In [ ]:
print('coefficient r_sq for rf is ', r2_score(y_train,ytrain_pred2))
print('mean absolute error for rf is ', mean_absolute_error(y_train,ytrain_pred2))
print('mean squared error for rf is ', mean_squared_error(y_train,ytrain_pred2))
print('explained variance score for rf is %.2f', explained_variance_score(y_train,ytrain_pred2))

In [ ]:
ytest_pred = dt.predict(X_test)
ytest_pred2 = rf.predict(X_test)

In [ ]:
X_test.shape

In [ ]:
y_test.head()

In [ ]:
y_test_dummy = y_test.copy()

In [ ]:
y_test = y_test[['Extra']]

In [ ]:
print('coefficient r_sq for decision tree is %.2f'% dt.score(X_test,y_test))
print('coefficient r_sq for random forest is %.2f'% rf.score(X_test,y_test))

In [ ]:
print('coefficient r_sq for decision tree is %.2f'% r2_score(y_test,ytest_pred))
print('coefficient r_sq for random forest is %.2f'% r2_score(y_test,ytest_pred2))

In [ ]:
print('Adjusted r_sq dt',1 - (1-dt.score(X_test, y_test))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1))
print('Adjusted r_sq rf',1 - (1-rf.score(X_test, y_test))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1))

In [ ]:
y_test_dummy['Predicted_dt'] = ytest_pred
y_test_dummy['Predicted_rf'] = ytest_pred2

In [ ]:
y_test_dummy.head()

In [ ]:
y_test_dummy.describe()

In [ ]:
rcParams['figure.figsize'] = 15, 15
import matplotlib.pyplot as plt
mm=21*24*60
fig = plt.figure()
ax = fig.add_subplot(111)
heatmap, xedges, yedges = np.histogram2d(x=y_test_dummy['Predicted_dt'],y=y_test_dummy['Actual_Extra'],weights=y_test_dummy['Totalres'] ,bins=200,range=[[0,mm],[0,mm]])
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
print(heatmap.min(),heatmap.max())
cutoff=10
pcm=plt.pcolormesh(xedges, yedges, heatmap, cmap='inferno',norm=colors.LogNorm(vmin=heatmap.min()+cutoff, vmax=heatmap.max()+cutoff))
plt.xlabel('Real timelimit and actual job length difference in seconds')
plt.ylabel('Predicted timelimit and actual job length difference in seconds')
plt.xlim(0,mm)
plt.ylim(0,mm)
plt.axis('square')
plt.title('Decision tree : 2D histogram for the predicted vs. actual run time')
fig.colorbar(pcm, ax=ax, extend='max')
if not os.path.exists('classification'):
    os.makedirs('classification')
plt.savefig('classification/Hypertuned_dt3_new_features.png')

In [ ]:
rcParams['figure.figsize'] = 15, 15
import matplotlib.pyplot as plt
mm=21*24*60
fig = plt.figure()
ax = fig.add_subplot(111)
heatmap, xedges, yedges = np.histogram2d(x=y_test_dummy['Predicted_rf'],y=y_test_dummy['Actual_Extra'],weights=y_test_dummy['Totalres'] ,bins=200,range=[[0,mm],[0,mm]])
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
print(heatmap.min(),heatmap.max())
cutoff=10
pcm=plt.pcolormesh(xedges, yedges, heatmap, cmap='inferno',norm=colors.LogNorm(vmin=heatmap.min()+cutoff, vmax=heatmap.max()+cutoff))
plt.xlabel('Real timelimit and actual job length difference in seconds')
plt.ylabel('Predicted timelimit and actual job length difference in seconds')
plt.xlim(0,mm)
plt.ylim(0,mm)
plt.axis('square')
plt.title('Random forest : 2D histogram for the predicted vs. actual run time')
fig.colorbar(pcm, ax=ax, extend='max')
if not os.path.exists('classification'):
    os.makedirs('classification')
plt.savefig('classification/hypertuned_rf3_new_features.png')

In [ ]:
y_test['predicted'] = ytest_pred2
y_test['Timelimit'] = X_test['TimelimitRaw'].to_numpy()

In [ ]:
y_test['y_actual'] = y_test_dummy['Actual_Extra']

In [ ]:
i = 0
predicted = []
extra = []
timelimit = []
Actual_Extra = []
df = pd.DataFrame(columns = ['Predicted','Extra','timelimit','Actual_Extra'])
for index, row in y_test.iterrows():
    if row['predicted'] > row['y_actual']:
        i = i + 1
        predicted.append(row['predicted'])
        extra.append(row['Extra'])
        timelimit.append(row['Timelimit'])
        Actual_Extra.append(row['y_actual'])

In [ ]:
df['Predicted'] = predicted
df['Extra'] = extra
df['timelimit'] = timelimit
df['Actual_Extra'] = Actual_Extra

In [ ]:
print(i)
df.tail(20)

## Approach 2 to predict the difference

In [ ]:
Approach2 = regression2[['Account','NNodes','Partition','TimelimitRaw','JobLen','Totalres']]

In [ ]:
Approach2.head()

In [ ]:
Approach2['TimelimitRaw'] = Approach2['TimelimitRaw'] / 60 
Approach2['JobLen'] = Approach2['JobLen'] / 60 

In [ ]:
Approach2['Extra'] = Approach2['TimelimitRaw'] - Approach2['JobLen']

In [ ]:
Approach2.describe()

In [ ]:
X = Approach2[['Account','NNodes','Partition','TimelimitRaw']]
y = Approach2[['Extra','Totalres']]


In [ ]:
y['Actual_Extra'] = y['Extra']

In [ ]:
label_encoder = preprocessing.LabelEncoder()
X['Partition']= label_encoder.fit_transform(X['Partition'])
X['Partition'].unique()

In [ ]:
y_dummy = y.copy()

In [ ]:
y = y[['Extra']]

In [ ]:
dt_a2 = DecisionTreeRegressor(random_state = 42).fit(X,y)

In [ ]:
rf_a2 = RandomForestRegressor(random_state = 42).fit(X,y)

In [ ]:
print('coefficient r_sq for decision tree is %.2f'% dt_a2.score(X,y))
print('coefficient r_sq for random forest is %.2f'% rf_a2.score(X,y))

In [ ]:
print('Adjusted r_sq dt',1 - (1-dt_a2.score(X,y))*(len(y)-1)/(len(y)-X.shape[1]-1))
print('Adjusted r_sq rf',1 - (1-rf_a2.score(X,y))*(len(y)-1)/(len(y)-X.shape[1]-1))

In [ ]:
ytrain_pred = dt_a2.predict(X)
ytrain_pred2 = rf_a2.predict(X)

In [ ]:
print('coefficient r_sq for dt is ', r2_score(y,ytrain_pred))
print('mean absolute error for dt is ', mean_absolute_error(y,ytrain_pred))
print('mean squared error for dt is ', mean_squared_error(y,ytrain_pred))
print('explained variance score for dt is %.2f', explained_variance_score(y,ytrain_pred))

In [ ]:
print('coefficient r_sq for rf is ', r2_score(y,ytrain_pred2))
print('mean absolute error for rf is ', mean_absolute_error(y,ytrain_pred2))
print('mean squared error for rf is ', mean_squared_error(y,ytrain_pred2))
print('explained variance score for rf is %.2f', explained_variance_score(y,ytrain_pred2))

In [ ]:
X['predicted_decision'] = ytrain_pred
X['predicted_random'] = ytrain_pred2

In [ ]:
X['Extra'] = y['Extra']

In [ ]:
X.head()

In [ ]:
Over_predicted = X.loc[X.predicted_decision > X.Extra] 
Over_predicted.info()

In [ ]:
def I(row):
    if row['predicted_decision'] >= row['Extra']:
        val = 1
    else:
        val = 0
    return val
X['Estimation_Logistic'] = X.apply(I,axis =1)

In [ ]:
X.describe()

In [ ]:
X.head()

In [ ]:
y['Totalres'] = y_dummy['Totalres']
y['Estimation_Logistic'] = X['Estimation_Logistic']
y.head(2)

In [ ]:
X = X[['Account','NNodes','Partition','TimelimitRaw','predicted_decision']]

In [ ]:
from sklearn import preprocessing
X[['NNodes', 'TimelimitRaw','predicted_decision']] = preprocessing.MinMaxScaler().fit_transform(X[['NNodes', 'TimelimitRaw','predicted_decision']])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y['Estimation_Logistic'], test_size=0.25,random_state = 42)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
y_train_dummy = y_train.copy()

In [ ]:
y_train = y_train[['Estimation_Logistic']]

In [ ]:
y_train.describe()

In [ ]:
y_tesdedt = y_test.loc[y_test.Estimation_Logistic == 0]

In [ ]:
y_tesdedt.info()

In [ ]:
classifier = DecisionTreeClassifier(criterion = 'entropy',random_state = 42)
classifier.fit(X_train,y_train)

In [ ]:
y_test_dummy = y_test.copy()

In [ ]:
y_test = y_test[['Estimation_Logistic']]

In [ ]:
y_pred = classifier.predict(X_test)


In [ ]:
y_test.head()

In [ ]:
sss = y_test

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)

In [ ]:
cm

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test,y_pred)
accuracy

In [ ]:
group_names = ['underestimated Actual and predicted ','underestimated Actual but predicted overestimated','overestimated Actual but predicted underestimated ','overestimated Actual and predicted']

group_counts = ["{0:0.0f}".format(value) for value in

                cm.flatten()]

group_percentages = ["{0:.2%}".format(value) for value in

                     cm.flatten()/np.sum(cm)]

labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in

          zip(group_names,group_counts,group_percentages)]

labels = np.asarray(labels).reshape(2,2)
np.sum(cm, axis=1, keepdims=False)
sns.set(rc = {'figure.figsize':(10,8)})
sns.heatmap(cm, annot=labels, fmt='', cmap='Blues',annot_kws={"size": 8})
if not os.path.exists('classification'):
    os.makedirs('classification')
plt.savefig('classification/dt_cm.png')

In [ ]:
group_names = ['underestimated Actual and predicted ','underestimated Actual but predicted overestimated','overestimated Actual but predicted underestimated ','underestimated Actual and predicted']

group_counts = ["{0:0.0f}".format(value) for value in

                cm.flatten()]

group_percentages = ["{0:.2%}".format(value) for value in

                     cm.flatten()/np.sum(cm)]

labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in

          zip(group_names,group_counts,group_percentages)]

labels = np.asarray(labels).reshape(2,2)

sns.heatmap(cm, annot=labels, fmt='', cmap='Blues')

if not os.path.exists('classification'):
    os.makedirs('classification')
plt.savefig('classification/dt_cm.png')

In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(random_state = 42)
log_reg.fit(X_train,y_train)

In [ ]:
y_pred = log_reg.predict(X_test)


In [ ]:
cm = confusion_matrix(y_test,y_pred)
cm

In [ ]:
accuracy = accuracy_score(y_test,y_pred)
accuracy

In [ ]:
X.head()

In [ ]:
group_names = ['underestimated Actual and predicted ','underestimated Actual but predicted overestimated','overestimated Actual but predicted underestimated ','overestimated Actual and predicted']

group_counts = ["{0:0.0f}".format(value) for value in

                cm.flatten()]

group_percentages = ["{0:.2%}".format(value) for value in

                     cm.flatten()/np.sum(cm)]

labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in

          zip(group_names,group_counts,group_percentages)]

labels = np.asarray(labels).reshape(2,2)
np.sum(cm, axis=1, keepdims=False)
sns.set(rc = {'figure.figsize':(10,8)})
sns.heatmap(cm, annot=labels, fmt='', cmap='Blues',annot_kws={"size": 8})
if not os.path.exists('classification'):
    os.makedirs('classification')
plt.savefig('classification/lr_cm.png')

In [ ]:
dt = DecisionTreeRegressor(random_state = 42).fit(X_train,y_train)

In [ ]:
rf = RandomForestRegressor(random_state = 42).fit(X_train,y_train)

In [ ]:
print('coefficient r_sq for decision tree is %.2f'% dt.score(X_train,y_train))
print('coefficient r_sq for random forest is %.2f'% rf.score(X_train,y_train))

In [ ]:
print('Adjusted r_sq dt',1 - (1-dt.score(X_train, y_train))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1))
print('Adjusted r_sq rf',1 - (1-rf.score(X_train, y_train))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1))

In [ ]:
ytrain_pred = dt.predict(X_train)
ytrain_pred2 = rf.predict(X_train)

In [ ]:
print('coefficient r_sq for dt is ', r2_score(y_train,ytrain_pred))
print('mean absolute error for dt is ', mean_absolute_error(y_train,ytrain_pred))
print('mean squared error for dt is ', mean_squared_error(y_train,ytrain_pred))
print('explained variance score for dt is %.2f', explained_variance_score(y_train,ytrain_pred))

In [ ]:
print('coefficient r_sq for rf is ', r2_score(y_train,ytrain_pred2))
print('mean absolute error for rf is ', mean_absolute_error(y_train,ytrain_pred2))
print('mean squared error for rf is ', mean_squared_error(y_train,ytrain_pred2))
print('explained variance score for rf is %.2f', explained_variance_score(y_train,ytrain_pred2))

In [ ]:
ytest_pred = dt.predict(X_test)
ytest_pred2 = rf.predict(X_test)

In [ ]:
X_test.shape

In [ ]:
y_test.head()

In [ ]:
y_test_dummy = y_test.copy()

In [ ]:
print('coefficient r_sq for decision tree is %.2f'% dt.score(X_test,y_test))
print('coefficient r_sq for random forest is %.2f'% rf.score(X_test,y_test))

In [ ]:
print('coefficient r_sq for decision tree is %.2f'% r2_score(y_test,ytest_pred))
print('coefficient r_sq for random forest is %.2f'% r2_score(y_test,ytest_pred2))

In [ ]:
print('Adjusted r_sq dt',1 - (1-dt.score(X_test, y_test))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1))
print('Adjusted r_sq rf',1 - (1-rf.score(X_test, y_test))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1))

In [ ]:
y_test_dummy['Predicted_dt'] = ytest_pred
y_test_dummy['Predicted_rf'] = ytest_pred2

In [ ]:
y_test_dummy.head()

In [ ]:
y_test_dummy['Predicted_dt'] = np.floor(y_test_dummy['Predicted_dt'])

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y_test_dummy['Estimation_Logistic'],y_test_dummy['Predicted_dt'])

In [ ]:
cm

In [ ]:
group_names = ['Overestimated Actual and predicted ','Overestimated Actual but predicted underestimated','Underestimated Actual but predicted Overestimated ','Underestimated Actual and predicted']

group_counts = ["{0:0.0f}".format(value) for value in

                cm.flatten()]

group_percentages = ["{0:.2%}".format(value) for value in

                     cm.flatten()/np.sum(cm)]

labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in

          zip(group_names,group_counts,group_percentages)]

labels = np.asarray(labels).reshape(2,2)

sns.heatmap(cm, annot=labels, fmt='', cmap='Blues')

## Approach 3 - time limit > 10 minutes

In [ ]:
Approach3 = regression2[['Account','NNodes','Partition','TimelimitRaw','JobLen','Totalres']]

In [ ]:
Approach3.head()

In [ ]:
Approach3['TimelimitRaw'] = Approach3['TimelimitRaw'] / 60 
Approach3['JobLen'] = Approach3['JobLen'] / 60 

In [ ]:
Approach3['Extra'] = Approach3['TimelimitRaw'] - Approach3['JobLen']

In [ ]:
Approach3.describe()

In [ ]:
negatives = Approach3.loc[Approach3.Extra <0]

In [ ]:
negatives.info()

In [ ]:
aa = Approach3.loc[Approach3.TimelimitRaw < 20]
aa.info()

In [ ]:
Approach3 = Approach3.loc[Approach3.TimelimitRaw > 20]

In [ ]:
X = Approach3[['Account','NNodes','Partition','TimelimitRaw']]
y = Approach3[['Extra','Totalres']]


In [ ]:
y['Actual_Extra'] = y['Extra']

In [ ]:
label_encoder = preprocessing.LabelEncoder()
X['Partition']= label_encoder.fit_transform(X['Partition'])
X['Partition'].unique()

In [ ]:
y_dummy = y.copy()

In [ ]:
y = y[['Extra']]

In [ ]:
dt_a3 = DecisionTreeRegressor(random_state = 42).fit(X,y)

In [ ]:
rf_a3 = RandomForestRegressor(random_state = 42).fit(X,y)

In [ ]:
print('coefficient r_sq for decision tree is %.2f'% dt_a3.score(X,y))
print('coefficient r_sq for random forest is %.2f'% rf_a3.score(X,y))

In [ ]:
print('Adjusted r_sq dt',1 - (1-dt_a3.score(X,y))*(len(y)-1)/(len(y)-X.shape[1]-1))
print('Adjusted r_sq rf',1 - (1-rf_a3.score(X,y))*(len(y)-1)/(len(y)-X.shape[1]-1))

In [ ]:
ytrain_pred = dt_a3.predict(X)
ytrain_pred2 = rf_a3.predict(X)

In [ ]:
print('coefficient r_sq for dt is ', r2_score(y,ytrain_pred))
print('mean absolute error for dt is ', mean_absolute_error(y,ytrain_pred))
print('mean squared error for dt is ', mean_squared_error(y,ytrain_pred))
print('explained variance score for dt is %.2f', explained_variance_score(y,ytrain_pred))

In [ ]:
print('coefficient r_sq for rf is ', r2_score(y,ytrain_pred2))
print('mean absolute error for rf is ', mean_absolute_error(y,ytrain_pred2))
print('mean squared error for rf is ', mean_squared_error(y,ytrain_pred2))
print('explained variance score for rf is %.2f', explained_variance_score(y,ytrain_pred2))

In [ ]:
X['predicted_decision'] = ytrain_pred
X['predicted_random'] = ytrain_pred2

In [ ]:
X['Extra'] = y['Extra']

In [ ]:
X.head()

In [ ]:
Over_predicted = X.loc[X.predicted_decision > X.Extra] 
Over_predicted.info()

In [ ]:
def I(row):
    if row['predicted_decision'] >= row['Extra']:
        val = 1
    else:
        val = 0
    return val
X['Estimation_Logistic'] = X.apply(I,axis =1)

In [ ]:
X.describe()

In [ ]:
X.head()

In [ ]:
y['Totalres'] = y_dummy['Totalres']
y['Estimation_Logistic'] = X['Estimation_Logistic']
y.head(2)

In [ ]:
X = X[['Account','NNodes','Partition','TimelimitRaw','predicted_decision']]

In [ ]:
from sklearn import preprocessing
X[['NNodes', 'TimelimitRaw','predicted_decision']] = preprocessing.MinMaxScaler().fit_transform(X[['NNodes', 'TimelimitRaw','predicted_decision']])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=X['Partition'], test_size=0.30,random_state = 42)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
y_train_dummy = y_train.copy()

In [ ]:
y_train = y_train[['Estimation_Logistic']]

In [ ]:
y_train.describe()

In [ ]:
classifier = DecisionTreeClassifier(criterion = 'entropy',random_state = 42)
classifier.fit(X_train,y_train)

In [ ]:
y_test_dummy = y_test.copy()

In [ ]:
y_test = y_test[['Estimation_Logistic']]

In [ ]:
y_pred = classifier.predict(X_test)


In [ ]:
y_test.head()

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)

In [ ]:
cm

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test,y_pred)
accuracy

In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(random_state = 42)
log_reg.fit(X_train,y_train)

In [ ]:
y_pred = log_reg.predict(X_test)


In [ ]:
cm = confusion_matrix(y_test,y_pred)
cm

In [ ]:
accuracy = accuracy_score(y_test,y_pred)
accuracy

In [ ]:
X.head()

In [ ]:
dt = DecisionTreeRegressor(random_state = 42).fit(X_train,y_train)

In [ ]:
rf = RandomForestRegressor(random_state = 42).fit(X_train,y_train)

In [ ]:
print('coefficient r_sq for decision tree is %.2f'% dt.score(X_train,y_train))
print('coefficient r_sq for random forest is %.2f'% rf.score(X_train,y_train))

In [ ]:
print('Adjusted r_sq dt',1 - (1-dt.score(X_train, y_train))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1))
print('Adjusted r_sq rf',1 - (1-rf.score(X_train, y_train))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1))

In [ ]:
ytrain_pred = dt.predict(X_train)
ytrain_pred2 = rf.predict(X_train)

In [ ]:
print('coefficient r_sq for dt is ', r2_score(y_train,ytrain_pred))
print('mean absolute error for dt is ', mean_absolute_error(y_train,ytrain_pred))
print('mean squared error for dt is ', mean_squared_error(y_train,ytrain_pred))
print('explained variance score for dt is %.2f', explained_variance_score(y_train,ytrain_pred))

In [ ]:
print('coefficient r_sq for rf is ', r2_score(y_train,ytrain_pred2))
print('mean absolute error for rf is ', mean_absolute_error(y_train,ytrain_pred2))
print('mean squared error for rf is ', mean_squared_error(y_train,ytrain_pred2))
print('explained variance score for rf is %.2f', explained_variance_score(y_train,ytrain_pred2))

In [ ]:
ytest_pred = dt.predict(X_test)
ytest_pred2 = rf.predict(X_test)

In [ ]:
X_test.shape

In [ ]:
y_test.head()

In [ ]:
y_test_dummy = y_test.copy()

In [ ]:
print('coefficient r_sq for decision tree is %.2f'% dt.score(X_test,y_test))
print('coefficient r_sq for random forest is %.2f'% rf.score(X_test,y_test))

In [ ]:
print('coefficient r_sq for decision tree is %.2f'% r2_score(y_test,ytest_pred))
print('coefficient r_sq for random forest is %.2f'% r2_score(y_test,ytest_pred2))

In [ ]:
print('Adjusted r_sq dt',1 - (1-dt.score(X_test, y_test))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1))
print('Adjusted r_sq rf',1 - (1-rf.score(X_test, y_test))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1))

In [ ]:
y_test_dummy['Predicted_dt'] = ytest_pred
y_test_dummy['Predicted_rf'] = ytest_pred2

In [ ]:
y_test_dummy.head()

In [ ]:
y_test_dummy['Predicted_dt'] = np.floor(y_test_dummy['Predicted_dt'])

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y_test_dummy['Estimation_Logistic'],y_test_dummy['Predicted_dt'])

In [ ]:
cm

In [ ]:
group_names = ['Overestimated Actual and predicted ','Overestimated Actual but predicted underestimated','Underestimated Actual but predicted Overestimated ','Underestimated Actual and predicted']

group_counts = ["{0:0.0f}".format(value) for value in

                cm.flatten()]

group_percentages = ["{0:.2%}".format(value) for value in

                     cm.flatten()/np.sum(cm)]

labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in

          zip(group_names,group_counts,group_percentages)]

labels = np.asarray(labels).reshape(2,2)

sns.heatmap(cm, annot=labels, fmt='', cmap='Blues')

## One hot encoding features

In [ ]:
regression3 = prediction2[['Account','NNodes','Partition','TimelimitRaw','JobLen','Totalres']]

In [ ]:
regression3['TimelimitRaw'] = regression3['TimelimitRaw'] * 60

In [ ]:
one_hot_encoded_data = pd.get_dummies(regression3, columns = ['Account', 'Partition'])
one_hot_encoded_data.head(10)

In [ ]:
X = one_hot_encoded_data.loc[:, one_hot_encoded_data.columns != 'JobLen']
X = X.loc[:, X.columns != 'Totalres']
y = one_hot_encoded_data[['JobLen','Totalres']]

In [ ]:
X.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25,random_state = 42)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
y_train_dummy = y_train.copy()

In [ ]:
y_train = y_train[['JobLen']]

In [ ]:
dt = DecisionTreeRegressor(random_state = 42).fit(X_train,y_train)

In [ ]:
rf = RandomForestRegressor(random_state = 42).fit(X_train,y_train)

In [ ]:
print('coefficient r_sq for decision tree is %.2f'% dt.score(X_train,y_train))
print('coefficient r_sq for random forest is %.2f'% rf.score(X_train,y_train))

In [ ]:
ytrain_pred = dt.predict(X_train)
ytrain_pred2 = rf.predict(X_train)

In [ ]:
print('coefficient r_sq for dt is ', r2_score(y_train,ytrain_pred))
print('mean absolute error for dt is ', mean_absolute_error(y_train,ytrain_pred))
print('mean squared error for dt is ', mean_squared_error(y_train,ytrain_pred))
print('explained variance score for dt is %.2f', explained_variance_score(y_train,ytrain_pred))

In [ ]:
print('coefficient r_sq for rf is ', r2_score(y_train,ytrain_pred2))
print('mean absolute error for rf is ', mean_absolute_error(y_train,ytrain_pred2))
print('mean squared error for rf is ', mean_squared_error(y_train,ytrain_pred2))
print('explained variance score for rf is %.2f', explained_variance_score(y_train,ytrain_pred2))

In [ ]:
y_test_dummy = y_test.copy()

In [ ]:
y_test = y_test[['JobLen']]

In [ ]:
y_test.head()

In [ ]:
ytest_pred = dt.predict(X_test)
ytest_pred2 = rf.predict(X_test)

In [ ]:
X_test.shape

In [ ]:
print('coefficient r_sq for decision tree is %.2f'% dt.score(X_test,y_test))
print('coefficient r_sq for random forest is %.2f'% rf.score(X_test,y_test))

In [ ]:
print('coefficient r_sq for decision tree is %.2f'% r2_score(y_test,ytest_pred))
print('coefficient r_sq for random forest is %.2f'% r2_score(y_test,ytest_pred2))

In [ ]:
rcParams['figure.figsize'] = 100, 20
x_ax = range(len(X_test))
plt.plot(x_ax,y_test,label = 'actual',color = 'k',linestyle = '-')
plt.scatter(x_ax,ytest_pred,label = 'predicted',color = 'r',linestyle = '--')
if not os.path.exists('classification'):
    os.makedirs('classification')
plt.savefig('classification/dt3.png')

In [ ]:
rcParams['figure.figsize'] = 100, 20
x_ax = range(len(X_test))
plt.plot(x_ax,y_test,label = 'actual',color = 'k',linestyle = '-')
plt.scatter(x_ax,ytest_pred2,label = 'predicted',color = 'r',alpha = 0.5,linestyle = '--')
if not os.path.exists('classification'):
    os.makedirs('classification')
plt.savefig('classification/rf3.png')

In [ ]:
y_test_dummy['Predicted_dt'] = ytest_pred
y_test_dummy['Predicted_rf'] = ytest_pred2

In [ ]:
rcParams['figure.figsize'] = 15, 15
import matplotlib.pyplot as plt
mm=21*24*3600
fig = plt.figure()
ax = fig.add_subplot(111)
heatmap, xedges, yedges = np.histogram2d(x=y_test_dummy['Predicted_dt'],y=y_test_dummy['JobLen'],weights=y_test_dummy['Totalres'] ,bins=200,range=[[0,mm],[0,mm]])
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
print(heatmap.min(),heatmap.max())
cutoff=10
pcm=plt.pcolormesh(xedges, yedges, heatmap, cmap='inferno',norm=colors.LogNorm(vmin=heatmap.min()+cutoff, vmax=heatmap.max()+cutoff))
plt.xlabel('Actual job length ')
plt.ylabel('Predicted')
plt.xlim(0,mm)
plt.ylim(0,mm)
plt.axis('square')
plt.title('k')
fig.colorbar(pcm, ax=ax, extend='max')
if not os.path.exists('classification'):
    os.makedirs('classification')
plt.savefig('classification/dt3_new.png')

In [ ]:
rcParams['figure.figsize'] = 15, 15
import matplotlib.pyplot as plt
mm=21*24*3600
fig = plt.figure()
ax = fig.add_subplot(111)
heatmap, xedges, yedges = np.histogram2d(x=y_test_dummy['Predicted_rf'],y=y_test_dummy['JobLen'],weights=y_test_dummy['Totalres'] ,bins=200,range=[[0,mm],[0,mm]])
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
print(heatmap.min(),heatmap.max())
cutoff=10
pcm=plt.pcolormesh(xedges, yedges, heatmap, cmap='inferno',norm=colors.LogNorm(vmin=heatmap.min()+cutoff, vmax=heatmap.max()+cutoff))
plt.xlabel('Actual job length ')
plt.ylabel('Predicted')
plt.xlim(0,mm)
plt.ylim(0,mm)
plt.axis('square')
plt.title('k')
fig.colorbar(pcm, ax=ax, extend='max')
if not os.path.exists('classification'):
    os.makedirs('classification')
plt.savefig('classification/rf3_new.png')

In [ ]:
y_test['predicted'] = ytest_pred2
y_test['Timelimit'] = X_test['TimelimitRaw'].to_numpy()

In [ ]:
i = 0
predicted = []
joblen = []
timelimit = []
df = pd.DataFrame(columns = ['Predicted','JobLen','timelimit'])
for index, row in y_test.iterrows():
    if row['predicted'] < row['JobLen']:
        i = i + 1
        predicted.append(row['predicted'])
        joblen.append(row['JobLen'])
        timelimit.append(row['Timelimit'])

In [ ]:
X_test.info()

In [ ]:
df['Predicted'] = predicted
df['JobLen'] = joblen
df['timelimit'] = timelimit


In [ ]:
print(i)
df.head(20)

## Linear Regression

In [ ]:
ml = LinearRegression().fit(X_train,y_train)

In [ ]:
print('coefficient r_sq for Linear Regression is %.2f'% ml.score(X_train,y_train))

In [ ]:
ytest_pred = ml.predict(X_test)

In [ ]:
print('coefficient r_sq for linear regression is %.2f'% r2_score(y_test,ytest_pred))

## Logistic Regression

In [ ]:
ml2 = SVR().fit(X_train,y_train)

In [ ]:
print('coefficient r_sq for SVM is %.2f'% ml2.score(X_train,y_train))

In [ ]:
ytest_pred = ml2.predict(X_test)

In [ ]:
print('coefficient r_sq for svm is %.2f'% r2_score(y_test,ytest_pred))